In [11]:
import pandas as pd
import numpy as np

**Sebelum Lanjut!,   jalankan "Librari Wrapper-Forward" di bagian paling bawah**

In [12]:
#data_cleanU1.xlsx
data = pd.read_excel("https://github.com/lufias69/Spam_Detection/blob/master/Filter%20chi%20square/komentar/data_cleanU1.xlsx?raw=true")
komentar = data.komentar.tolist()
label = data.label.tolist()

y = np.array(label)
print(len(y))

data

5047


,Column1,label,komentar
0,195,spam,ga nyesel deh aku order sana
1,480,non spam,wakakak tolol tulang tulang
2,539,non spam,sepatu keren kamu keren cara instan
3,564,spam,via q udh blja shoppe barang bagus kecewa mantap
4,596,non spam,sir what if use again kamisenglish that s just...
...,...,...,...
5042,5049,spam,bosen outfit kalian aja yuk cek ig tambah kole...
5043,5050,spam,maantaap banggeet kaak eteaah akuu pakee gemuk...
5044,5052,spam,sllu ajar usaha tahan kualitas costumer neng s...
5045,5053,spam,aasi ane o a uda nenain odu yan sana eui eecay...


In [ ]:
#ini lama bangets
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
hasil = wrapper(model=MultinomialNB(), weighting=TfidfVectorizer, corpus=komentar, y=y,n_splits=5, target=0.97)

$.

In [1]:
best_fitur = hasil[1]

for i in best_fitur:
    print(i)

NameError: ignored

**Librari Wrapper-Forward**

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import StratifiedKFold

def convert_bin(data, key=False):
    new_data = list()
    if (type(data)==list or type(data)==str) and key==False:
        if type(data)==str:
            for d in data:
                new_data.append(int(d))
        else:
            for d in data:
                new_data.append(int(d))
        return new_data
    else:
        for d in data:
            new_data.append(str(d))
        return "".join(new_data)
            
def join_bin(data1, data2):
    new_data = list()
    for a,b in zip(data1, data2):
        if a==b:
            new_data.append(a)
        elif a!=b:
            if b==1:
                new_data.append(b)
            elif a==1:
                new_data.append(a)
            else:
                new_data.append(0)
    return new_data

def gen(x, urutan):
    first_bin = [0 for i in range(0,x)]
    first_bin[urutan]=1
    return first_bin

def generate_bins(n):
    bin_all = list()
    for i in range(0,n):
        bin_gen = gen(n,i)
        bin_all.append(bin_gen)
    return bin_all

def convert_to_index(data):
    new_data = list()
    for idx, i in enumerate(data):
        if i!=0:
            new_data.append(idx)
    return new_data



def wrapper(model=None, weighting=TfidfVectorizer(), corpus=None, y=None,n_splits=5, target=0.97):
    
    y=np.array(y)
    vectorizer = weighting()
    X = vectorizer.fit_transform(corpus)
    fitur = np.array(vectorizer.get_feature_names())
    panjang_fitur = len(fitur)
    
    binari_pertama = generate_bins(panjang_fitur)
    scor_binari_pertama = list()
    print("$",end=".")
    for b in binari_pertama:
        idx_bin = convert_to_index(b)
        # print(fitur[idx_bin],end="")
        vectorizer = weighting(vocabulary=fitur[idx_bin])
        X = vectorizer.fit_transform(corpus)
        
        skf = StratifiedKFold(n_splits=n_splits)
        skor_list = list()
        
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            model.fit(X_train, y_train)
            skor_list.append(model.score(X_test,y_test))
            
        scor_binari_pertama.append(sum(skor_list)/len(skor_list))
    print("^^",end=".")  
    max_index = scor_binari_pertama.index(max(scor_binari_pertama))
    pilihan_pertama_bin = binari_pertama[max_index]
    print(max(scor_binari_pertama))
    
    all_bin = list()
    all_skor = list()
    
    for _ in range(panjang_fitur-1):
        print(_, end=".")
        new_bin_ = list()
        for idx, i in enumerate(binari_pertama):
            if idx !=pilihan_pertama_bin:
                new_bin_.append(join_bin(convert_bin(pilihan_pertama_bin, key=True), i)) 
                
        binari_pertama = list(new_bin_)
                
#fdsf
        scor_binari_pertama = list()
        for b in new_bin_:
            idx_bin = convert_to_index(b)
            vectorizer = weighting(vocabulary=fitur[idx_bin])
            X = vectorizer.fit_transform(corpus)

            skf = StratifiedKFold(n_splits=n_splits)
            skor_list = list()
            for train_index, test_index in skf.split(X, y):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                model.fit(X_train, y_train)
                skor_list.append(model.score(X_test,y_test))

            scor_binari_pertama.append(sum(skor_list)/len(skor_list))
        #
        max_index = scor_binari_pertama.index(max(scor_binari_pertama))
        pilihan_pertama_bin = convert_bin(new_bin_[max_index], key=False) 
        print(max(scor_binari_pertama))
        
        all_bin += list(new_bin_)
        all_skor += list(scor_binari_pertama)
        
        if max(scor_binari_pertama) >= target:
            return (max(scor_binari_pertama), fitur[convert_to_index(pilihan_pertama_bin)])
        elif sum(pilihan_pertama_bin)>= panjang_fitur-1:
            return (max(scor_binari_pertama), fitur[convert_to_index(pilihan_pertama_bin)])
        
    max_index = all_skor.index(max(all_skor))
    pilihan_pertama_bin =  convert_bin(all_bin[max_index], key=False) 
    return (max(all_skor), fitur[convert_to_index(pilihan_pertama_bin)])
    

In [9]:
convert_bin([1,0,1,1,1], key=True)

'10111'

In [3]:
convert_bin('10111', key=False)

[1, 0, 1, 1, 1]